# Lecture10 動的システム
<div dir='rtl'>
2024.5岩政
</div>

## ステップ応答（時間応答）
1次系のステップ応答，インパルス応答  
2次系のインパルス応答  

最初にステップ応答を、微分方程式を数値的に解くことで求めます。数値積分関数scipy.integrate.odeintを用います。

scipy.integrate.odeint https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html

In [ ]:
# -*- coding: utf-8 -*-
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### 1次系のステップ応答
1次のODE (Ordinary Differential Equation)

積分すべき微分方程式を定義しておきます。

ここでは、

$$
 \frac{dx(t)}{dt}=-a\times x(t)+u(t)
$$

であるとします。関数は 信号変数x(t),時刻刻み、そして、入力パラメータ、信号を引数とします。

またa=2.0とします。

In [ ]:
def dFunc_1(x, time, a, u):
    dx = -a*x + u
    return dx

time = np.linspace(0,3,100) # time interval, 100 division
a = 2.0

初期値x(0)=0として、入力としてはu(t)=1というステップ関数を準備します。

そして、odeintに対して、微分方程式（関数で定義した）、初期値、時間刻み、パラメータ、入力信号を渡します。

odeintの結果プロットすると、微分方程式を積分した結果、すなわちステップ応答が得られます。

In [ ]:
x0 = 0.0 # Initial value
u = 1.0  # Step input
sol_1 = odeint(dFunc_1, x0 ,time, args=(a,u))
plt.plot(time, sol_1, '-k', linewidth=2)
plt.xlabel('t', fontsize=20)
plt.ylabel('x', fontsize=20, rotation='horizontal')
plt.grid()


#### 1次系のインパルス応答

今度はインパルス応答を求めてみます。インパル信号を時間領域(tということ)で具体化するのは困難なので、時刻=0からずっと0となる入力信号をインパルス応答の代わりとします。

In [ ]:
x0 = 1.0 # Initial value
u = 0.0  # zero input
sol_2 = odeint(dFunc_1, x0 ,time, args=(a,u))
plt.plot(time, sol_2, '-b', linewidth=2)
plt.xlabel('t', fontsize=20)
plt.ylabel('x', fontsize=20, rotation='horizontal')
plt.grid()


#### 2次系のステップ応答
質量(mass)，ばね定数（spring）の係数は一定として，ダンパ係数を変えてシミュレーションを実施

In [ ]:
def dFunc_2(x, time, mass, damper, spring, u):
    dx1 = x[1]
    dx0 = (-1/mass)*(damper*x[1] + spring*x[0] - u) 
    return [dx1, dx0]

time = np.linspace(0,20,100)
u = 1.0  # input
x0 = [0.0, 0.0]

mass, damper, spring = 4.0, 0.4, 1.0 # damper; changeable

sol_1 = odeint(dFunc_2, x0, time, args=(mass, 1.0, spring, u))
sol_2 = odeint(dFunc_2, x0, time, args=(mass, 2.0, spring, u))
sol_3 = odeint(dFunc_2, x0, time, args=(mass, 4.0, spring, u))
sol_4 = odeint(dFunc_2, x0, time, args=(mass, 6.0, spring, u))

plt.plot(time, sol_1[:,[0]], label='D=1')
plt.plot(time, sol_2[:,[0]], label='D=2')
plt.plot(time, sol_3[:,[0]], label='D=4')
plt.plot(time, sol_4[:,[0]], label='D=6')

plt.xlabel('t')
#plt.ylabel('x', rotation='horizontal')
plt.ylabel('x')
plt.legend()
plt.grid()


## 伝達関数による記述

'control'パッケージを利用します。

In [ ]:
#インストールが必要かも。
!pip install control

In [ ]:
from control.matlab import *

伝達関数モデルの記述

ここでは、伝達関数
$$
\mathcal{P}(s)=\frac{1}{s^2+2s+3}
$$

をtf(分子、分母)により作成します。

In [ ]:
Np = [0, 1]      # 伝達関数の分子多項式の係数 (0*s + 1)
Dp = [1, 2, 3]   # 伝達関数の分母多項式の係数 (1*s^2 + 2*s + 3)
P = tf(Np, Dp)
print('P(s)=', P)

直接分子、分母を与えてもできます。

In [ ]:
P = tf([0, 1], [1, 2, 3])
print('P(s)=', P)

#### ステップ応答

step()関数を使って、伝達関数に対するステップ応答を得ます。

便利関数です。

In [ ]:
def cross_lines(x, y, **kwargs):
    ax = plt.gca()
    ax.axhline(y, **kwargs)
    ax.axvline(x, **kwargs)
    ax.scatter(T, 0.632, **kwargs)

伝達関数でモデルを定義して、ステップ応答をみてみます。tf([0, 1], [0.5, 1])とは

$$
 \mathcal{P}(s)=\frac{1}{0.5s+1}
$$

一次系の伝達関数が

$$G(s)=\frac{K}{Ts+1}$$

- Tは時定数(time constraint)、Kは定常ゲイン(steady-state gain)と呼ばれる
- ステップ応答は、単位ステップ信号のラプラス変換が$1/s$であることから、

Kが1、Tが0.5となります。t=0.5のときに、K*0.632=0.632に立ち上がるはずです。

In [ ]:
from control.matlab import * #︓伝達関数モデルの定義
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(figsize=(3, 2.3))
P = tf([0, 1], [0.5, 1])
y, t = step(P, np.arange(0, 5, 0.01))
T=0.5
ax.plot(t, y, color='k')
cross_lines(T, 0.632, color='k',lw=0.5)
ax.annotate('$(0.5, 0.632)$', xy=(0.7, 0.5))
ax.set_xticks(np.linspace(0, 5, 6))
ax.grid(ls=':')
#stepinfo(P)

時間応答

In [ ]:
from control.matlab import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#便利関数
def plot_set(fig_ax, *args):
    fig_ax.set_xlabel(args[0])
    fig_ax.set_ylabel(args[1])
    fig_ax.grid(ls=':')
    if len(args)==3:
        fig_ax.legend(loc=args[2])

def linestyle_generator():
    linestyle = ['-', '--', '-.', ':']
    lineID = 0
    while True:
        yield linestyle[lineID]
        lineID = (lineID + 1) % len(linestyle)

#### １次系
一次遅れ系（再掲）


$$G(s)=\frac{K}{Ts+1}$$

- Tは時定数(time constraint)、Kは定常ゲイン(steady-state gain)と呼ばれる
- ステップ応答は、単位ステップ信号のラプラス変換が$1/s$であることから、

In [ ]:
def cross_lines(x, y, **kwargs):
    ax = plt.gca()
    ax.axhline(y, **kwargs)
    ax.axvline(x, **kwargs)
    ax.scatter(T, 0.632, **kwargs)
    
fig, ax = plt.subplots(figsize=(3, 2.3))

(T, K) = (0.5, 1)
P = tf([0, K], [T, 1])
y, t = step(P, np.arange(0, 5, 0.01))
ax.plot(t,y, color='k')

cross_lines(T, 0.632, color='k',lw=0.5)
ax.annotate('$(0.5, 0.632)$', xy=(0.7, 0.5))

ax.set_xticks(np.linspace(0, 5, 6))
plot_set(ax, 't', 'y')
ax.set_title('Step Response(T=0.5,K=1)')

Tを色々変えてみると、反応の速度がかわることがわかります。

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2.3))
LS = linestyle_generator()

K = 1
T = (1, 0.5, 0.1)
for i in range(len(T)):
    y, t = step(tf([0, K], [T[i], 1]), np.arange(0, 5, 0.01))
    ax.plot(t, y, ls = next(LS), label='T='+str(T[i]))


ax.set_xticks(np.linspace(0, 5, 6))
ax.set_yticks(np.linspace(0, 1, 6))
plot_set(ax, 't', 'y', 'best')

#### 2次系

２次系の、伝達関数は、

$$
G(s)=K\frac{\omega_n^2}{s^2+2\zeta \omega_n s+\omega_n^2}
$$


- K=1、$\zeta=0.4,\omega_n=5$の場合のステップ応答
- 時刻Tpで、最大値ymaxとなる
  - $Tp=\pi/(\omega_n\sqrt{1-\zeta^2})$
  - $y_{max}=K(1+e^{-\zeta \omega_n T_p})$


In [ ]:
def cross_lines(x, y, **kwargs):
    plt.gca()
    ax.axhline(y, **kwargs)
    ax.axvline(x, **kwargs)
    ax.scatter(x, y, **kwargs)
    
(zeta, omega_n) = (0.4, 5)

fig, ax = plt.subplots(figsize=(3, 2.3))

P = tf([0,omega_n**2], [1, 2*zeta*omega_n, omega_n**2])
y, t = step(P, np.arange(0,5,0.01))
ax.plot(t,y, color='k')

ymax = 1 + 1 * np.exp(-(np.pi*zeta)/np.sqrt(1-zeta**2))
Tp = np.pi/omega_n/np.sqrt(1-zeta**2)
cross_lines(Tp, ymax, color='k',lw=0.5)

ax.annotate('$(T_P, y_{max})$', xy=(1.2, 1.1))

print('ymax=',ymax)
print('Tp=', Tp)

ax.set_xticks(np.arange(0, 5.2, step=1.0))
ax.set_yticks(np.arange(0, 1.3, step=0.25))
plot_set(ax, 't', 'y')
ax.set_title('Step Response(zeta=0.4,omega_n=5)')


- $\zeta$の値でステップ応答がどう変わるか（$\omega_n$は速応性をきめる)
  - $\zeta \gt 0$: 過減衰、$\zeta =1$: 臨界減衰
  - $0 \lt \zeta \lt 1$: 減衰振動、$\zeta=0$:持続振動、$\zeta \lt 0$:発散

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2.3))

LS = linestyle_generator()

zeta = [2,1, 0.7, 0.4]
omega_n = 5
for i in range(len(zeta)):
    P = tf([0, omega_n**2], [1, 2*zeta[i]*omega_n, omega_n**2])
    y, t = step(P, np.arange(0, 5, 0.01))
    
    pltargs = {'ls': next(LS), 'label': '$\zeta$='+str(zeta[i]) }
    ax.plot(t, y, **pltargs)

ax.set_xticks(np.arange(0, 5.2, step=1.0))
ax.set_yticks(np.arange(0, 1.3, step=0.25))
plot_set(ax, 't', 'y', 'best')    

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2.3))

LS = linestyle_generator()

zeta = [0.1, 0, -0.05]
omega_n = 5
for i in range(len(zeta)):
    P = tf([0, omega_n**2], [1, 2*zeta[i]*omega_n, omega_n**2])
    y, t = step(P, np.arange(0, 5, 0.01))
    
    pltargs = {'ls': next(LS), 'label': '$\zeta$='+str(zeta[i])}
    ax.plot(t, y, **pltargs)
 
ax.set_xticks(np.arange(0, 5.2, step=1.0))
ax.set_yticks(np.arange(-4, 5, step=2))

plot_set(ax, 't', 'y', 'lower left')

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2.3))

LS = linestyle_generator()

zeta = 0.7
omega_n = [1, 5, 10]
for i in range(len(omega_n)):
    P = tf([0, omega_n[i]**2], [1, 2*zeta*omega_n[i], omega_n[i]**2])
    y, t = step(P, np.arange(0, 5, 0.01))
    
    pltargs = {'ls': next(LS)}
    pltargs['label'] = '$\omega_n$='+str(omega_n[i])
    ax.plot(t, y, **pltargs)

ax.set_xticks(np.arange(0, 5.2, step=1.0))
plot_set(ax, 't', 'y', 'best')

#### 状態空間モデルとステップ応答

- pythonによる状態空間モデルの表現とステップ応答を求める例
- $A=\begin{bmatrix} 0& 1\\-4 & -5\end{bmatrix},B=\begin{bmatrix} 0\\ 1\end{bmatrix},C=\begin{bmatrix} 1& 0\\0 & 1\end{bmatrix},D=\begin{bmatrix} 0\\ 0\end{bmatrix}$としたときの応答


In [ ]:
A = [[0, 1],[-4, -5]]
B = [[0], [1]]
C = np.eye(2)
D = np.zeros([2, 1])
P = ss(A, B, C, D)

Td = np.arange(0, 5, 0.01)
x, t = step(P, Td) #ゼロ状態応答

fig, ax = plt.subplots()
ax.plot(t, x[:,0], label='$x_1$')
ax.plot(t, x[:,1], ls='-.', label='$x_2$')
ax.set_xticks(np.linspace(0, 5, 6))
ax.set_yticks(np.linspace(-0.4, 0.6, 6))
plot_set(ax,'t','x','best')

#### 安定性
- 伝達関数から極、零点、
- 極(pole):伝達関数の分母の根(root)、全ての極の実部が負ならば安定
- 'sys'パッケージのpole(モデル)関数を用いて極を求められる。


In [ ]:
P1 = tf([0,1],[1, 1])
print('P1:', pole(P1))
P2 = tf([0,1],[-1, 1])
print('P2:', pole(P2))
P3 = tf([0,1],[1, 0.05, 1])
print('P3:', pole(P3))
P4 = tf([0,1],[1, -0.05, 1])
print('P4:', pole(P4))

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))

P1pole = pole(P1)
P2pole = pole(P2)
P3pole = pole(P3)
P4pole = pole(P4)
ax.scatter(P1pole.real, P1pole.imag, s=50, marker='o',label='$P_1$', color='k')
ax.scatter(P2pole.real, P2pole.imag, s=50, marker='^',label='$P_2$', color='k')
ax.scatter(P3pole.real, P3pole.imag, s=50, marker='x',label='$P_3$', color='k')
ax.scatter(P4pole.real, P4pole.imag, s=50, marker='*',label='$P_4$', color='k')

ax.set_xlim(-1.2,1.2)
ax.set_ylim(-1.2,1.2)
plot_set(ax, 'Re', 'Im', 'best')

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
y, t = step(P1, np.arange(0,5,0.01))
ax.plot(t,y, color='k')
ax.set_xticks(np.linspace(0, 5, 6))
plot_set(ax, 't', 'y')
ax.set_title('P1 = tf([0,1],[1, 1])')

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
y, t = step(P2, np.arange(0,5,0.01))
ax.plot(t,y, color='k')
ax.set_xticks(np.linspace(0, 5, 6))
plot_set(ax, 't', 'y')
ax.set_title('P2 = tf([0,1],[-1, 1])')

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
y, t = step(P3, np.arange(0,20,0.01))
ax.plot(t,y, color='k')
ax.set_xticks(np.linspace(0, 20, 21))
plot_set(ax, 't', 'y')
ax.set_title('tf([0,1],[1, 0.05, 1])')

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
y, t = step(P4, np.arange(0,20,0.01))
ax.plot(t,y, color='k')
ax.set_xticks(np.linspace(0, 20, 21))
plot_set(ax, 't', 'y')
ax.set_title('tf([0,1],[1, -0.05, 1])')

## ラプラス変換・逆変換

### ラプラス変換
'sympy'パッケージの laplace_transform関数を用いてラプラス変換を実行できます。

laplace_transform(関数 ,t,s)

例えば、 定数関数は $\frac{1}{s}$に変換されます。ここでt,sは最初にSymbolとして定義する必要があります。

In [ ]:
import sympy as sp
s = sp.Symbol('s')
t = sp.Symbol('t', positive=True)
sp.init_printing()
sp.laplace_transform(1, t, s)[0]

In [ ]:
alpha=sp.Symbol('alpha')
sp.laplace_transform(sp.exp(alpha*t),t,s)[0]

ラプラス変換テーブルを、実際に、まとめてやってみる

In [ ]:
import sympy as sp
sp.init_printing()
t, s = sp.symbols('t, s')
a = sp.symbols('a', real=True, positive=True)
omega = sp.Symbol('omega', real=True)
exp = sp.exp
sin = sp.sin
cos = sp.cos
functions = [1,
         t,
         exp(-a*t),
         t*exp(-a*t),
         t**2*exp(-a*t),
         sin(omega*t),
         cos(omega*t),
         1 - exp(-a*t),
         exp(-a*t)*sin(omega*t),
         exp(-a*t)*cos(omega*t),
         ]
functions

In [ ]:
def L(f):
    return sp.laplace_transform(f, t, s, noconds=True)

In [ ]:
[L(f) for f in functions]

#### 1次系のステップ応答を解く

制御対象$\mathcal{P}(s)$の出力は、$y(s)=\mathcal{P}(s)u(s)$

ステップ入力の場合は$u(s)=\frac{1}{s}$になるので、$y(s)=\frac{\mathcal{P}(s)}{s}$となる

１次遅れ系の場合は、

$$
y(s)=\frac{K}{Ts+1}\frac{1}{s}=K\Big(\frac{1}{s}-\frac{T}{1+Ts}\Big)=k\Big( \frac{1}{s}-\frac{1}{s+\frac{1}{T}} \Big)
$$

$y(T)=1-e^{-1}=0.632$である

'sympy.apart'は部分分数展開を得る

In [ ]:
import sympy as sp
sp.init_printing()
s= sp.Symbol('s')
T= sp.Symbol('T',real=True)
P = 1/((1+T*s)*s)
sp.apart(P,s)

また逆ラプラス変換は

In [ ]:
import sympy as sp
sp.init_printing()
s=sp.Symbol('s')
t=sp.Symbol('t', positive=True)
T=sp.Symbol('T',real=True)
sp.inverse_laplace_transform(1/s-1/(s+1/t),s,t)

２つをまとめると

In [ ]:
import sympy as sp
sp.init_printing()
#sp.init_printing(False, str_printer=lambda x: sp.latex(x))
s= sp.Symbol('s')
t=sp.Symbol('t', positive=True)
T= sp.Symbol('T',real=True)
P = 1/(1+T*s)*(1/s) # １次系のステップ応答
Papart=sp.apart(P,s) # 部分分数展開
print(Papart)
ret= sp.inverse_laplace_transform(Papart,s,t) #逆ラプラス変換
ret.subs([(t,T)])# tにTを代入

２次系のステップ応答は、$\zeta=1$のときは、

$$
K\frac{\omega_n^2}{s^2+2\omega_n s+\omega_n^2}=K\frac{\omega_n^2}{s(s+\omega_n)^2}
$$

これを部分分数分解すると（ラプラス変換しやすくするため）

$$
y(s)=K\Big(\frac{1}{s}-\frac{1}{s+\omega_n}-\frac{\omega_n}{(s+\omega_n)^2}\Big)
$$

となり、これを逆ラプラス変換して

$$
 y(t)=K\Big(1-e^{-\omega_n t}-\omega_n te^{-\omega_n t}\Big)
$$

となる、これより$y(0)=0,y(\infin)=K$であることがわかる。

この変換をSympyでおkなう。

In [ ]:
import sympy as sp
sp.init_printing()
s=sp.Symbol('s')
t=sp.Symbol('t', positive=True)
w=sp.Symbol('w',real=True)
P=sp.apart(w**2/s/(s+w)**2,s)
sp.inverse_laplace_transform(P,s,t)